In [93]:
import string
from os import listdir
import os
import errno
import subprocess
from collections import defaultdict
import operator
import re
import pickle
from collections import OrderedDict
import six # needed for Google Cloud client

import unidecode

import numpy as np
import pandas as pd
import scipy.sparse

from scipy.sparse.csr import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import model_from_json

In [13]:
df = pd.read_json('recipes/'+'ethnic_categories.json', dtype={})

In [14]:
df.sample(10)

,categories,cookingTime,description,ethnicCategory,fullCategoryTree,ingredients,instructionSteps,name,rating,ratingCount,url
1075,"[Soup, Soups, Stews and Chili, Japanese Recipe...",PT30M,NaN,japanese,asian/japanese,"[4 cups water, 1/2 cup bonito flakes, 1 (4 inc...",[Heat water in a large pot over low heat. Add ...,Authentic Miso Soup,4.81,8,http://allrecipes.com/recipe/231545/authentic-...
14195,"[German Recipes, Kirschwasser Desserts, Liqueu...",PT2H10M,NaN,german,european/german,"[1 2/3 cups all-purpose flour, 2/3 cup unsweet...",[Preheat oven to 350 degrees F (175 degrees C)...,Black Forest Cake II,4.32,142,http://allrecipes.com/recipe/8444/black-forest...
7767,"[Canadian Cookies, International Cookies, Coco...",PT30M,NaN,canadian,canadian,"[1 1/2 cups all-purpose flour, 1 1/2 teaspoons...",[Preheat oven to 375 degrees F (190 degrees C)...,Chocolate Chip and Coconut Lunchbox Cookies,4.73,10,http://allrecipes.com/recipe/234222/chocolate-...
11002,"[Italian Recipes, Ribs, Everyday Cooking, Main...",PT2H,NaN,italian,european/italian,"[1 tablespoon butter, 3 slices bacon, chopped,...",[Preheat oven to 350 degrees F (175 degrees C)...,Italian Style Short Ribs,4.21,11,http://allrecipes.com/recipe/26613/italian-sty...
8526,"[Vegetarian Gravy, Gravy, Sauces and Condiment...",PT30M,NaN,us-recipes,us-recipes,"[1/2 cup vegetable oil, 1/3 cup chopped onion,...",[Heat oil in a medium saucepan over medium hea...,Vegetarian Gravy,4.59,316,http://allrecipes.com/recipe/13799/vegetarian-...
2213,"[Mexican Recipes, Main Dishes, Recipes, Mexica...",NaN,NaN,mexican,latin-american/mexican,"[2 tablespoons olive oil, 1 large onion, chopp...",[Heat oil in a 12-inch skillet until shimmerin...,Southwestern-Flavored Ground Beef or Turkey fo...,4.38,220,http://allrecipes.com/recipe/76984/southwester...
3739,"[Onion Side Dishes, Vegetable Side Dishes, Pot...",PT1H5M,NaN,us-recipes,us-recipes,"[20 pods okra, sliced in 1/4 inch pieces, 6 Yu...","[Place vegetables in a colander, rinse with co...",Southern Style Fried Okra,4.13,12,http://allrecipes.com/recipe/177248/southern-s...
8623,"[Broccoli Salad, Vegetable Salads, Salad Recip...",PT2H25M,NaN,canadian,canadian,"[3 cups broccoli florets, 1/2 cup chopped red ...","[In a measuring cup, mix together yogurt, mayo...",Broccoli Buffet Salad,4.67,43,http://allrecipes.com/recipe/74166/broccoli-bu...
7086,"[Pork Spare Ribs, Pork Ribs, Ribs, Main Dishes...",PT12H,NaN,us-recipes,us-recipes,"[6 pounds pork spareribs, Dry rub:, 1/2 cup pa...","[In a medium bowl, mix together the brown suga...",Not Your Every Day Smoked Pork Spare Ribs,4.68,84,http://allrecipes.com/recipe/85634/not-your-ev...
1718,"[Indian Recipes, Shrimp Recipes, Seafood Curry...",PT30M,This is a subtle yet richly flavored curry tha...,indian,asian/indian,"[2 tablespoons peanut oil, 1/2 sweet onion, mi...",[Heat the oil in a large skillet over medium h...,Indian Shrimp Curry,4.43,155,http://allrecipes.com/recipe/205544/indian-shr...


In [15]:
df[['url']].values[5883]

array([u'http://allrecipes.com/recipe/26671/texas-cowboy-chili-beans/'], dtype=object)

In [16]:
#df[['ethnicCategory']].values.nunique()

df.groupby('ethnicCategory').ethnicCategory.count()

ethnicCategory
argentinian                       24
australian-and-new-zealander     499
austrian                          30
bangladeshi                       15
belgian                           15
brazilian                         93
canadian                        1030
chilean                           32
chinese                          323
colombian                         15
cuban                             81
czech                             22
danish                            21
dutch                             44
east-african                      30
egyptian                          29
english                          143
filipino                         147
finnish                           19
french                           440
german                           301
greek                            377
hungarian                         42
indian                           671
indonesian                        33
irish                            199
israeli                

In [17]:
print 'Total {0}'.format(len(df))

Total 14442


In [18]:
cat_mapping_lines = """argentinian  latin_american
australian-and-new-zealander  north_european
austrian  north_european
bangladeshi  south_asian
belgian  north_european
brazilian  latin_american
canadian  north_american
chilean  latin_american
chinese  east_asian
colombian  latin_american
cuban  latin_american
czech  east_european
danish  north_european
dutch  north_european
east-african  african
egyptian  african
english  north_european
filipino  southeast_asian
finnish  north_european
french  north_european
german  north_european
greek  middle_eastern
hungarian  east_european
indian  south_asian
indonesian  southeast_asian
irish  north_european
israeli  middle_eastern
italian  south_european
jamaican  latin_american
japanese  east_asian
korean  east_asian
lebanese  middle_eastern
malaysian  southeast_asian
mexican  latin_american
moroccan  african
norwegian  north_european
pakistani  south_asian
persian  middle_eastern
peruvian  latin_american
polish  east_european
portuguese  south_european
puerto-rican  latin_american
russian  east_european
scottish  north_european
south-african  african
spanish  south_european
swedish  north_european
swiss  north_european
thai  southeast_asian
turkish  middle_eastern
us-recipes  north_american
vietnamese  southeast_asian
welsh  north_european"""

#print cat_mapping_lines
cat_mapping_dict = {}
cat_mapping_list = cat_mapping_lines.split('\n')
#print cat_mapping_list

for mapping in cat_mapping_list:
    #print line
    smaller_category, larger_category = mapping.split()
    cat_mapping_dict[smaller_category] = larger_category
print cat_mapping_dict

{'australian-and-new-zealander': 'north_european', 'portuguese': 'south_european', 'irish': 'north_european', 'mexican': 'latin_american', 'chinese': 'east_asian', 'czech': 'east_european', 'scottish': 'north_european', 'east-african': 'african', 'japanese': 'east_asian', 'moroccan': 'african', 'spanish': 'south_european', 'persian': 'middle_eastern', 'polish': 'east_european', 'austrian': 'north_european', 'swedish': 'north_european', 'israeli': 'middle_eastern', 'puerto-rican': 'latin_american', 'turkish': 'middle_eastern', 'south-african': 'african', 'bangladeshi': 'south_asian', 'german': 'north_european', 'peruvian': 'latin_american', 'indian': 'south_asian', 'dutch': 'north_european', 'korean': 'east_asian', 'pakistani': 'south_asian', 'colombian': 'latin_american', 'belgian': 'north_european', 'danish': 'north_european', 'indonesian': 'southeast_asian', 'hungarian': 'east_european', 'egyptian': 'african', 'welsh': 'north_european', 'brazilian': 'latin_american', 'us-recipes': 'n

In [19]:
def consolidate_ethnic_categories(cat):
    return cat_mapping_dict[cat]

In [20]:
df['consolidated_category'] = df['ethnicCategory'].apply(consolidate_ethnic_categories)

df.sample(25)

,categories,cookingTime,description,ethnicCategory,fullCategoryTree,ingredients,instructionSteps,name,rating,ratingCount,url,consolidated_category
1408,"[Sauces, Sauces and Condiments, Vietnamese Rec...",PT5M,NaN,vietnamese,asian/vietnamese,"[1 cup fish sauce, 3/4 cup rice vinegar, 3 tab...","[In a medium bowl, combine the fish sauce, ric...",Vietnamese Dipping Sauce,3.60,9,http://allrecipes.com/recipe/56926/vietnamese-...,southeast_asian
404,"[Indian Recipes, Asian Recipes, Shrimp Recipes...",PT30M,NaN,indian,asian/indian,"[1/2 pound prawns, peeled and deveined, sea sa...","[Season prawns with sea salt., Grind coconut, ...",Goan Prawn Pulao,5.00,1,http://allrecipes.com/recipe/245940/goan-prawn...,south_asian
9382,"[Mexican Recipes, Everyday Cooking, Main Dishe...",PT45M,"Very easy, yet very good wet burritos. I got ...",mexican,latin-american/mexican,"[1 pound ground beef, 1/2 cup chopped onion, 1...",[Crumble ground beef into a skillet over mediu...,Fabulous Wet Burritos,4.50,904,http://allrecipes.com/recipe/70404/fabulous-we...,latin_american
8877,"[Adult Punch, Punch, Drinks, Everyday Cooking,...",PT2H10M,NaN,canadian,canadian,"[4 cups cranberry juice cocktail, 8 cups prepa...","[In a large punch bowl, combine cranberry juic...",Holiday Punch I,4.57,79,http://allrecipes.com/recipe/21234/holiday-pun...,north_american
7561,"[Lamb Recipes, Steaks and Chops, Everyday Cook...",PT1H55M,NaN,australian-and-new-zealander,australian-and-new-zealander,"[1 tablespoon butter, 1 onion, sliced, 2 table...",[Preheat oven to 350 degrees F (175 degrees C)...,Plum Lamb Casserole,0.00,0,http://allrecipes.com/recipe/246684/plum-lamb-...,north_european
4614,"[Side Dishes, Noodles, Recipes, Hanukkah Kugel...",PT1H10M,NaN,us-recipes,us-recipes,"[4 cups egg noodles, 1 (8 ounce) package cream...",[Preheat oven to 350 degrees F (175 degrees C)...,Traditional Apple Noodle Kugel,1.33,2,http://allrecipes.com/recipe/235084/traditiona...,north_american
673,"[Indian Recipes, Iced Tea, Tea Drinks, Drinks,...",PT5M,NaN,indian,asian/indian,"[3/4 cup boiling water, 1 chai tea bag, 1 1/2 ...",[Pour boiling water over tea bag in a mug; let...,Chai Tea Latte,4.04,17,http://allrecipes.com/recipe/222513/chai-tea-l...,south_asian
918,"[Vegetable Soup, Chicken and Rice Soup, Chicke...",PT25M,NaN,chinese,asian/chinese,"[3 ounces baby shrimp, 3 ounces skinless, bone...","[Mix together the shrimp, chicken, egg, and co...",Chinese Sizzling Rice Soup,4.57,21,http://allrecipes.com/recipe/13125/chinese-siz...,east_asian
4352,"[Baked and Roasted Chicken, Vegetable Casserol...",PT1H15M,NaN,us-recipes,us-recipes,"[3 slices bacon, 16 canned beef tamales, cut i...","[In a skillet over medium heat, fry the bacon ...",Oklahoma Tamale Casserole,3.67,5,http://allrecipes.com/recipe/68292/oklahoma-ta...,north_american
11295,"[Pasta Sauces, Sauces, Sauces and Condiments, ...",PT8H,NaN,italian,european/italian,"[2 tablespoons olive oil, 3 whole garlic clove...",[Heat 2 tablespoons of olive oil over medium h...,Grandma's Old Italian Spaghetti Sauce with Mea...,3.71,7,http://allrecipes.com/recipe/191507/grandmas-o...,south_european


In [21]:
df.groupby('consolidated_category').consolidated_category.count()

consolidated_category
african             140
east_asian          662
east_european       267
latin_american     2517
middle_eastern      555
north_american     4534
north_european     1938
south_asian         714
south_european     2606
southeast_asian     509
Name: consolidated_category, dtype: int64

In [41]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()&*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        #print "unicode instance"
        translate_table = dict((ord(char), unicode(translate_to)) for char in not_letters)
    else:
        #print "not unicode instance"
        assert isinstance(to_translate, str)
        #translate_table = string.maketrans(not_letters, translate_to *len(not_letters))
        return to_translate.translate(None, string.punctuation + string.digits)
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list, strip_punct=True):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl', 'fluid',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'jiggers', 'jigger',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'coating', 'coatings',
        'carton', 'cartons',
        'count',
        'dusting', 'dustings',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'each',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'balls',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'packet', 'packets',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'sleeve', 'sleeves',
        'rounds',
        'sheet', 'sheets',
        'squares',
        'semicircles', 'semicircle',
        'circles', 'circle',
        'moons', 'halfmoons',
        'strands', 'strands',
        'ribbons', 'ribbon',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'fourth', 'fourths',
        'quarter', 'quarters',
        'fifth', 'fifths',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten', 'wellbeaten',
        'blanched',
        'blended',
        'boiled',
        'bottled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'converted',
        'cooked',
        'cooled',
        'cored',
        'crispcooked',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'deshelled',
        'desilked',
        'deveined',
        'diced', 'dice',
        'dissolved',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'husked',
        'individual',
        'jarred',
        'juiced',
        'julienned', 'julienne', 'juliennecut', 'juliennesliced',
        'kneaded', 'kneading',
        'marinated',
        'mashed',
        'matchstickcut',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'pounded',
        'prepared',
        'pressed',
        'prewashed',
        'processed',
        'pulled',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'rolled', 'rolling',
        'salted',
        'saved',
        'scored',
        'scrubbed',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shelled',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'snipped',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'strained',
        'soaked',
        'sweetened',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'undrained',
        'unhusked',
        'unopened',
        'unpeeled',
        'unsalted',
        'unsweetened',
        'unwashed',
        'unwrapped',
        'use', 'used',
        'warmed',
        'washed',
        'wrapped',
        'zested',
    ]
    
    other_words_to_filter = [
        # articles:
        'a', 'an', 'the',
        # adverbs:
        'accordingly', 'according',
        'all',
        'approximately',
        'artificially ',
        'coarsely',
        'crisply',
        'crosswise',
        'diagonally', 
        'finely',
        'freshly',
        'horizontally',
        'left',
        'lengthwise', 
        'lightly',
        'like',
        'moreorless ',
        'nearly',
        'overnight',
        'piecewise',
        'plus',
        'right',
        'roughly',
        'sideways',
        'slightly',
        'stiffly',
        'then',
        'thickly',
        'thinly',
        'vertically',
        'very',
        'well',
        'widthwise', 
        # adjectives:
        'additional ',
        'aged',
        'allpurpose',
        'another',
        'any',
        'best',
        'certified',
        'chunky',
        'classic',
        'coarse', 
        'cold',
        'complete',
        'crisp',
        'diagonal',
        'discarded',
        'dry',
        'extralarge',
        'extrasmall', 
        'even',
        'fatfree',
        'favourite', 'favorite',
        'fine', 
        'firm',
        'flat',
        'fresh',
        'hard',
        'hot',
        'inchesthick', 'inchthick',
        'included',
        'inner',
        'jumbo',
        'large', 
        'lean',
        'leftover',
        'less',
        'long',
        'lowcalorie',
        'lowfat',
        'lowsodium',
        'matchsticksize',
        'medium',
        'moist',
        'more',
        'multicolored',
        'natural',
        'needed',
        'new',
        'nonfat',
        'old',
        'oldfashioned',
        'only',
        'optional',
        'organic',
        'original',
        'other',
        'outer',
        'pure',
        'real',
        'reduced',
        'reserved',
        'rough',
        'sharp',
        'short',
        'small',
        'stale',
        'soft',
        'such',
        'supreme',
        'thick',
        'thickcut',
        'thicklycut',
        'thicklysliced',
        'thicksliced',
        'thin',
        'thincut',
        'thinlycut',
        'thinlysliced',
        'thinsliced',
        'tough',
        'young',
        'your',
        # prepositions:
        'about',
        'above',
        'across',
        'against',
        'apart',
        'as',
        'at',
        'away',
        'below',
        'by',
        'for',
        'from',
        'in',
        'into',
        'of',
        'off',
        'on',
        'out',
        'to',
        'until',
        'with',
        'without',
        # conjunctions:
        'and',
        'but',
        'if',
        'or',
        'so',
        # abbreviations:
        'eg', 'ie',
        # nouns:
        'accompaniments',
        'blossoms',
        'bottoms', 'bottom',
        'choice',
        'directions',
        'ends', 'end',
        'fists', 'fist',
        'hands', 'hand',
        'hours', 'hour',
        'ingredients',
        'list',
        'minutes', 'minute',
        'slicer',
        'peeler',
        'product',
        'purpose',
        'reserve',
        'room',
        'seconds', 'second',
        'spirals', 'spiral',
        'temperature',
        'thickness',
        'thinness',
        'tops', 'top',
        # verbs:
        'add',
        'alternating',
        'are',
        'be',
        'discarding',
        'including',
        'is',
        'sit',
        'using',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    if strip_punct:
        cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    else:
        cleaned_ingred_list = full_ingred_list
    
    #print "length of cleaned_ingred_list: {}".format(len(cleaned_ingred_list))
    #print cleaned_ingred_list
    if len(cleaned_ingred_list) == 0:
        print "length of cleaned_ingred_list is zero"
        print "full_ingred_list:"
        print full_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    if isinstance(cleaned_ingred_list[0], unicode):
        ingreds_only = [' '.join(w for w in unidecode.unidecode(ingred_item).split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    else:
        assert isinstance(cleaned_ingred_list[0], str)
        ingreds_only = [' '.join(w for w in ingred_item.split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    brand_names = [
        'a(r)', # A1 steak sauce
        'absolut(r)',
        'accents(r)', 'accents(tm)', 'accent(r)',
        'act ii(r)',
        'adams(r)',
        'al fresco(r)',
        'amafil(r)',
        'annies lemon chive dressing(r)',
        'archer farms(tm)',
        'arrowhead mills(r)',
        'artisan blends(r)',
        'aylmer(r) accents(r)',
        'babybel(r)',
        'bacardi (r)', 'bacardi(r)',
        'backdraft fire sauce(r)',
        'ball(r) kerr(r)',
        'ball park(r)',
        'barilla(r)',
        'barill pronto(tm)',
        'bay(r)', 'bay (tm)',
        'beau monde (tm)',
        'beechers(r) flagship', 'beechers flagship', 'beechers(r)',
        'beer shiner(r)',
        'belgioioso',
        'benedictine(r)',
        'bertolli(r)',
        'best foods(r)',
        'better than bouillon(r)',
        'betty crocker(r)',
        'birds eye(r)',
        'bisquick(r)',
        'bisquick (r)',
        'bobs red mill(r)',
        'borden(r)',
        'bragg(r)',
        'breeze(r)',
        'briannas home style(r)', 'briannas(r)',
        'brooks(r)',
        'bruces(r)',
        'budweiser(r)',
        'buitoni(r)',
        'bullseye(r)',
        'bumble bee(r)',
        'burnetts(r)',
        'bushs best(r)', 'bushs homestyle(r)', 'bushs chili magic(r)', 'bushs(r)',
        'butterball(r)',
        'butter buds(r)',
        'campari(r)',
        'campbells(r) healthy request', 'campbells(r)', 'campbells (r)',
        'carnation (r)',
        'carolina pride(r)',
        'carroll shelbys(r)',
        'cascadian farm(r)',
        'catelli bistro(r)',
        'cattlemens(r)',
        'cavenders(r)',
        'chambord(r)',
        'chef paul prudhommes salmon magic(r)',
        'cheese whiz(r)', 'cheez whiz(r)',
        'chobani(r)',
        'cholula(r)',
        'ciroc(r)',
        'clamato(r)',
        'classicmac(tm)',
        'classico (tm)',
        'classico (r)',
        'classico(r)',
        'claussen(r)',
        'coke(r)',
        'cointreau(r)',
        'college inn(r)',
        'contadina(r)',
        'cool whip(r)',
        'coors(r)',
        'cornnuts (r)',
        'corona(r)',
        'cortland',
        'country time(r)',
        'craisins(r)',
        'crazy steves(tm) cajun cukes', 'crazy steves(r)',
        'cremovo(r)',
        'crisco(r)',
        'daisy(r)',
        'dei fratelli(r)',
        'del monte(r)',
        'diamond crystal(r)',
        'dickinsons(r)',
        'dole(r)',
        'dole asian island crunch(tm)',
        'dole(r) veggie(tm)',
        'doritos(r)',
        'dubliner(r)',
        'dukes(r)',
        'duncan hines(r)',
        'durkee(r)', 'famous sauce(r)',
        'eagle brand(r)',
        'earth balance(r)',
        'eckrich(r)',
        'edward sons(r)',
        'egg beaters(r)',
        'el paso(r) ',
        'el pato(r)',
        'embasa(r)',
        'emerald(r)',
        'equal(r)',
        'farmland(r)',
        'fat tire(r)',
        'fiesta blend(r)',
        'fleischmanns(r)', 'fleischmanns ',
        'folgers(r)',
        'follow your heart(r) veganaise(r)', 'follow your heart(r)',
        'fosters (r)',
        'foster farms(r)',
        'franks(r) redhot', 'franks red hot (r)', 'franks redhot (r)', 'franks redhot(r)', 'redhot(r)', 'franks(r)',
        'frenchs(r)',
        'frichik(r)',
        'fritos(r)',
        'frontera(r)',
        'goya (r)', 'goya(r)',
        'gebhardt(r)',
        'ghirardelli(r)',
        'girards(r) olde venice',
        'good seasons(r)',
        'goodness(r)',
        'gourmet garden(tm)',
        'grapenuts(tm)',
        'great american spice co(tm)',
        'green giant(r)', 'green giant(tm) crumbles(r)', 'green giant(tm)',
        'grill mates(r)',
        'guinness(r)',
        'hangar one(r)',
        'harp(r)',
        'hatch(r)',
        'healthy choice(r)',
        'heineken(r)', 
        'heinz (r)', 'heinz(r)',
        "hellmann's(r) best foods(r)", 'hellmanns(r) best foods(r)', 'hellmanns(r)best foods(r)',
        'hellmanns light(r)', 'hellmans(r) light', 'hellmanns(r) low fat',
        'hellmans(r)', "hellmann's(r)", 'hellmanns(r)',
        'hershey(r)s', 'hershey(r)s kisses(r)', 'hersheys(r) chipits(r)', 'hersheys(r)',
        'hidden valley ranch(r)', 'hidden valley(r) original ranch(r)', 'hidden valley(r) ranch(r)', 'hidden valley(r)', 'hidden valley (r)',
        'hillshire farm(r)',
        'holland house(r)',
        'horizon(r)',
        'hormel(r)', 'hormel(tm)',
        'house tsang(r)',
        'hunts(r)',
        'idahoan(r)','baby reds(r)', 'buttery homestyle(r)',
        'imagine(r)', 'imagine(tm)',
        'isalys(r)',
        'jack daniels(r)',
        'jello(r)',
        'jennieo(r)',
        'jif(r)',
        'jiffy(r)',
        'jim beam(r)',
        'jimmy dean(r)',
        'johnsonville(r)',
        'johnnys seasoning salt(r)',
        'jose cuervo(r)', 'cuervo(r)',
        'kame(r)',
        'karo(r)',
        'karys(r)',
        'kc masterpiece(r)',
        'keebler club(r)', 'keebler(r)',
        'keens(r)',
        'kens steak house(r)',
        'kens steak house lite northern italian dressing(r)',
        'kerrygold(r)',
        'kewpie(r)',
        'kikkoman(r)',
        'king arthur(r)',
        'king oscar(r)',
        'kirkland(r)',
        'kitchen basics(r)',
        'kitchen bouquet(r)',
        'kitchen secrets(r)',
        'klondike gourmet(r)', 'klondike goldust(r)', 'klondike(r)',
        'knorr(r) rice sides(tm)', 'knorr(r) fiesta sides(tm)', 'knorr(r)', 'knorrs(r)',
        'knox (r)',
        'kraft(r)',
        'land o lakes(r)', 'land olakes(r)',
        'laphroiag(r)',
        'laughing cow(r)',
        'lawrys(r)',
        'libbys(r)',
        'lightlife(r) organic smoky tempeh strips(r)', 'lightlife(r)',
        'lipton(r) recipe secrets(r)', 'lipton(r)',
        'locatelli(r)',
        'loma linda(r)',
        'lotus(r)',
        'louisiana(r)',
        'maggi(r)',
        'magic(r)',
        'mahatma(r)',
        'maifun(r)',
        'maille(r)',
        'maldon(r)',
        'malibu(r)',
        'marie callenders(r)',
        'maries(r)',
        'market pantry(tm)',
        'martha white(r)',
        'mary kitchen(r)',
        'marzettis(r)',
        'maui(r)',
        'mazola(r)',
        'mccormick chili seasoning mix sodium(r)', 'mccormick(r)',
        'mexene(r)',
        'mexenes(r)',
        'mexicorn(r)',
        'miller lite(r)',
        'mimiccreme(r)',
        'minute(r)',
        'miracle whip(r)',
        'miracle whip(tm)',
        'miracle whip (tm)',
        'miracle whip free(r)',
        'miracle whip light(r)', 'light miracle whip(r)',
        'mission(r)',
        'montreal steak seasoning(r)',
        'morningstar farms(r) grillers recipe crumbles(r)', 'morningstar farms(r) recipe crumbles(r)', 'morningstar farms(r)',
        'morton(r) natures seasons(r)', 'mortons(r)', 'morton(r)',
        'motts(r)',
        'mountain dew(r)',
        'mrs butterworths(r)',
        'mrs dash(r)',
        'muir glen(r)',
        'nabisco(r)',
        'natural goodness(tm)',
        'negra modelo(r)',
        'nescafe(r)',
        'nestle(r) toll house(r)', 'nestle(r) carnation(r)', 'nestle(r)',
        'newcastle(r)',
        'newmans own(r)',
        'newmans own(r) lighten up(r)', 'lighten up(r)',
        'niblets(r)',
        'nissin(r) top ramen',
        'no yolks(r)',
        'ocean spray(r)',
        'oikos(r)',
        'old bay(r)', 'old bay (tm)', 'old bay(tm)',
        'old el paso(r)',
        'olive garden(r)',
        'oreida(r) steam n mash(r)', 'oreida(r)', 'ore ida(r)',
        'oreo(r)',
        'ortega(r)',
        'ottogi ramyonsari(r)', 'ottogi(r)',
        'pace(r)',
        'pam(r)',
        'parkay(r)',
        'pataks(r)',
        'penzeys(r)', 'penzys(r)',
        'pepperidge farm(r)',
        'perdue(r)',
        'pernod(r)',
        'perrins(r)',
        'pet(r)',
        'pillsbury grands(r)', 'pillsbury(r) grands(r)', 'pillsbury(r)',
        'planters(r)',
        'plochmans(r)',
        'prego(r)',
        'progresso(tm) recipe starters(tm)', 'progresso(r)',
        'quaker(r)',        
        'quorn(tm)',
        'ragu(r) old world style(r)', 'ragu(r)',
        'rapunzel(r)',
        'ready rice(r)',
        'realime(r)',
        'red gold(r)',
        'reddiwip(r)',
        'reeses(r)',
        'renees(r)',
        'reynolds wrap(r)', 'reynolds(r)',
        'rice sides(tm)',
        'ronzoni(r)',
        'rotel(r)',
        'rumford(r)',
        'saco(r)',
        'sambazon',
        'samuel smith(r)',
        'sargento(r)', 'sargento (r)',
        'sason accent(r)',
        'sb(r) golden curry', 'sb(r)',
        'seagrams(r)',
        'seasonall(r)',
        'shiner bock(r)',
        'silk(r)',
        'simple crisp(tm)',
        'simply balanced(tm)',
        'simply potatoes(r)',
        'skippy(r)',
        'skyline chili(r)',
        'slap ya mama(r)',
        'smart balance(r)',
        'smirnoff(r)',
        'smithfield(r)',
        'smuckers(r)',
        'snack factory(r)',
        'snickers(r)',
        'so delicious(r)',
        'southern comfort (r)',
        #'spam(r)',
        'spectrum naturals(r)', 'spectrum(r)',
        'spice islands(r)',
        'spike(r)',
        'splenda(r)',
        'sprite(r)',
        'sriracha(r)',
        'steam whistle pilsner(r)',
        'stovetop(r)',
        'sunmaid(r)',
        'sunsweet(r) dnoir(tm)',
        'sutton dodge(tm)',
        'swanson(r) vegetable flavor boost(r)', 'swanson(r)', 'natural goodness(r)',
        'sweet baby rays(r)',
        'sweetn low(r)',
        'tabasco(r)', 'tabasco(tm)',
        'taco bell(r)',
        'tajin(r)',
        'tapatio(r)',
        'tater tots(r)',
        'tenderflake(r)',
        'texas pete(r)', 'texas petess(r)',
        'thai kitchen(r)',
        'three bridges(r)',
        'tillamook(r)',
        'tony chacheres(r)',
        'top ramen(r)',
        'tostitos(r)',
        'trader joes(r)',
        'truroots(r)',
        'truvia(r)',
        'tyson(r) ready(r)', 'tyson(r)',
        'uncle bens (r)', 'uncle bens(r)',
        'up(r)', # 7-up
        'v(r)', # what remains from "V8"
        'valentina(r)',
        'vegenaise(r)',
        'veggie(tm)',
        #'velveeta(r)',
        'vernors(r)',
        'vidalia(r)',
        'vietti(r)',
        'vindaloo sauce maya kaimal(r)', 'vindaloo(r)',
        'voskos(r)',
        'watkins(r)',
        'wesson(r)',
        'white lily(r)',
        'wishbone(r)',
        'wolf brand (r)', 'wolf(r)',
        'wondra(r)',
        'yoplait(r)',
        'yuengling(r)',
        'yves veggie cuisine(r)',
        'zatarains(r)',
        'zing zang(r)',
    ]
    
    def remove_brands(ingred_str):
        str_without_brand = ingred_str
        for bn in brand_names:
            str_without_brand = str_without_brand.replace(bn, '')
        return str_without_brand
    
    ingreds_without_brands = [' '.join(remove_brands(ingred_str).strip().split()) for ingred_str in ingreds_only]
    
    def remove_duplicate_words(ingred_str):
        ingred_words = ingred_str.split()
        no_dupes_ingred_str = " ".join(sorted(set(ingred_words), key=ingred_words.index))
        return no_dupes_ingred_str
    
    ingreds_duplicates_removed = [remove_duplicate_words(ingred_str) for ingred_str in ingreds_without_brands]
    
    return ingreds_duplicates_removed


In [42]:
ingreds_arr_of_lists = np.squeeze(df[['ingredients']].values)
vect_extract_ingred = np.vectorize(extract_ingredients, otypes=[list])

processed_ingreds_arr_of_lists = vect_extract_ingred(ingreds_arr_of_lists)

print type(processed_ingreds_arr_of_lists)
print "Processed ingredients from {} recipes".format(len(processed_ingreds_arr_of_lists))
print processed_ingreds_arr_of_lists.size
print processed_ingreds_arr_of_lists.shape

<type 'numpy.ndarray'>
Processed ingredients from 14442 recipes
14442
(14442,)


In [43]:
with open('ethnic_ingreds_arr_of_lists.pkl', 'wb') as f:
    pickle.dump(ingreds_arr_of_lists, f)

with open('ethnic_processed_ingreds_arr_of_lists.pkl', 'wb') as f:
    pickle.dump(processed_ingreds_arr_of_lists, f)


In [44]:
flattened_ingred_list = np.hstack(processed_ingreds_arr_of_lists)
print len(flattened_ingred_list)
print flattened_ingred_list.size
print flattened_ingred_list.shape

144078
144078
(144078,)


In [45]:
unique_vocab = sorted(list(set(flattened_ingred_list)))
print unique_vocab
print "{} unique ingredients found".format(len(unique_vocab))

['absinthe', 'acai berry pulp', 'achar masala', 'achiote annatto seeds', 'achiote paste', 'achiote powder', 'achiote seed', 'achiote seeds', 'acini di pepe pasta', 'acorn squash', 'active yeast', 'additional butter margarine', 'additional butter sugar garnish', 'additional extravirgin olive oil brushing', 'additional green onions', 'additional heavy cream topping', 'additional parmesan cheese serving', 'additional ranch dressing', 'adobo sauce', 'adobo sauce chilies', 'adobo sauce chipotle peppers', 'adobo sauce chipotle peppers goy', 'adobo seasoning', 'adobo seasoning cumin', 'adobo seasoning goy', 'adobo seasoning pepper goy', 'adzuki bean filling', 'adzuki beans', 'agaragar flakes', 'agaragar powder', 'agave nectar', 'agave syrup', 'age argentinian white wine', 'ahi steaks', 'ahi tuna', 'ahi tuna steaks', 'ahi yellowfin tuna loin', 'ahi yellowfin tuna steak', 'ahi yellowfin tuna sushigrade', 'aji amarillo', 'aji amarillo chile paste', 'aji amarillo pepper', 'aji amarillo peppers', 

In [46]:
max(unique_vocab, key=len)

'assorted cutup vegetables broccoli carrots tomatoes bell peppers cauliflower onions mushrooms'

In [47]:
brand_name_ingreds = [s for s in unique_vocab if '(r)' in s or '(tm)' in s]
print len(brand_name_ingreds)
#print brand_name_ingreds
for brand_name in brand_name_ingreds:
    print brand_name

207
amaretto liqueur disaronno(r)
amber beer honey brown lager(r)
angostura bitters(r)
apricot fruit filling solo(r)
arepas flour pan(r)
aromatic bitters peychauds(r)
asian sweet chile sauce mae ploy(r) chilli
baby broccoli broccolini(r)
baking chocolate bakers(r)
ball(r) fiesta salsa mix before measuring
basil simply organic(r)
batter mix shore lunch(r)
bay seasoning(tm)
berries europes best(r) field berry mix
bitter orange aperitif aperol(r)
bittersweet chocolate baker's(r)
black red pepper blend mccormick shot(r)
blackened seasoning chef paul prudhommes(r) steak
bloody mary mix mr mrs t(r)
blue green fruitflavored candies skittles(r)
bonito soup stock hondashi(r)
bourbon bulleit(r)
bourbon four roses(r)
bran cereal kelloggs(r) allbran(r)
brandy presidente(r)
brandybased orange liqueur grand marnier(r)
bread cakes purity(r)
brown sugar butter flavored breakfast cereal capn crunch(r)
brown sugar butter flavored crispy corn oat breakfast cereal cap'n crunch(r)
buttery crackers ritz(r)


In [48]:
df['cleanedIngredients'] = df['ingredients'].apply(extract_ingredients)
df.sample(10)

,categories,cookingTime,description,ethnicCategory,fullCategoryTree,ingredients,instructionSteps,name,rating,ratingCount,url,consolidated_category,cleanedIngredients
3050,"[Mission®, Trusted Brands, Mexican Recipes, Ev...",PT20M,NaN,mexican,latin-american/mexican,[4 Mission® Kid Size Flour Tortillas with Edib...,[Spread 2 tablespoons of marinara sauce on two...,Mini Pizza-dillas,4.23,10,http://allrecipes.com/recipe/86011/mini-pizza-...,latin_american,[kid flour tortillas edible disney(r) tortilla...
10308,"[Seafood Appetizers, Appetizers and Snacks, It...",PT15M,NaN,italian,european/italian,"[2 cups vegetable oil for frying, 1 cup butter...",[Heat oil in a saucepan or deep-fryer to 375 d...,Buttermilk-Battered Calamari,4.38,63,http://allrecipes.com/recipe/69818/buttermilk-...,south_european,"[vegetable oil frying, buttermilk, flour, salt..."
13514,"[Scandinavian Recipes, Swedish Recipes, Spice ...",PT57M,NaN,swedish,european/scandinavian/swedish,"[1 cup butter, 3/4 cup white sugar, 1 teaspoon...",[Preheat oven to 350 degrees F (180 degrees C)...,Drommar,4.42,11,http://allrecipes.com/recipe/10015/drommar/,north_european,"[butter, white sugar, baking powder, vanilla e..."
9714,"[Mexican Recipes, Mexican-Style Chicken Breast...",PT45M,A great way to use leftover chicken. Even kids...,mexican,latin-american/mexican,"[1 tablespoon butter, 1/2 cup chopped green on...",[Preheat oven to 350 degrees F (175 degrees C)...,Chicken Enchiladas II,4.56,3096,http://allrecipes.com/recipe/8694/chicken-ench...,latin_american,"[butter, green onions, garlic powder, green ch..."
9608,"[Soup, Soups, Stews and Chili, Brazilian Recip...",PT13H15M,My version of this national Brazilian black be...,brazilian,latin-american/south-american/brazilian,"[1 pound dry black beans, 2 quarts water, plus...","[Place black beans into a large bowl, cover wi...",Chef John's Brazilian Feijoada,4.93,12,http://allrecipes.com/recipe/232437/chef-johns...,latin_american,"[black beans, water, beef, bay leaf, pork chop..."
10332,"[Scrambled Eggs, Breakfast Eggs, Breakfast and...",PT10M,NaN,italian,european/italian,"[1 tablespoon vegetable oil, 1 egg, lightly be...",[Heat oil in a skillet over medium heat. In a ...,Pesto Scrambled Eggs,4.40,57,http://allrecipes.com/recipe/47908/pesto-scram...,south_european,"[vegetable oil, egg, cheddar cheese, salt pepp..."
2750,"[Sweet Potato Side Dishes, Vegetable Side Dish...",PT20M,NaN,mexican,latin-american/mexican,"[6 small yams, 6 tablespoons butter, 6 teaspoo...",[Preheat oven to 250 degrees F (120 degrees C)...,Camotes al Horno (Baked Yams),4.41,19,http://allrecipes.com/recipe/17362/camotes-al-...,latin_american,"[yams, butter, honey, white sugar]"
9220,"[Sandwiches, Everyday Cooking, Main Dishes, Re...",PT50M,I came up with this recipe to entertain friend...,us-recipes,us-recipes,"[1/4 cup mayonnaise, 3 cloves garlic, minced, ...","[In a bowl, mix the mayonnaise, minced garlic,...",California Grilled Veggie Sandwich,4.81,815,http://allrecipes.com/recipe/14504/california-...,north_american,"[mayonnaise, garlic, lemon juice, olive oil, r..."
9242,"[Seafood Stew, Stew, Soups, Stews and Chili, I...",PT55M,A wonderful seafood stew! Serve with a loaf of...,australian-and-new-zealander,australian-and-new-zealander,"[3/4 cup butter, 2 onions, chopped, 2 cloves g...",[Over medium-low heat melt butter in a large s...,Cioppino,4.76,563,http://allrecipes.com/recipe/12816/cioppino/,north_european,"[butter, onions, garlic, parsley, stewed tomat..."
5993,"[Broiled Chicken Breasts, Chicken Breasts, Bak...",PT20M,NaN,us-recipes,us-recipes,"[4 skinless, boneless chicken breast halves, 1...","[Preheat oven to Broil., Place chicken in a fo...",Buffalo Chicken Sandwiches,4.24,32,http://allrecipes.com/recipe/20887/buffalo-chi...,north_american,"[skinless boneless chicken breast, pepper sauc..."


In [49]:
df.to_pickle('EthnicCleanedIngredients.pkl')

In [50]:
!aws s3 cp CleanedIngredients.pkl s3://RecipeVectors

upload: ./CleanedIngredients.pkl to s3://RecipeVectors/CleanedIngredients.pkl


In [52]:
cv2 = CountVectorizer(vocabulary=unique_vocab, analyzer=lambda x:x, tokenizer=lambda text: text, lowercase=False)
recipe_ingredient_matrix2 = cv2.fit_transform(np.squeeze(df[['cleanedIngredients']].values))

print type(recipe_ingredient_matrix2)
print recipe_ingredient_matrix2.size
print recipe_ingredient_matrix2.nnz
print recipe_ingredient_matrix2.shape

<class 'scipy.sparse.csr.csr_matrix'>
140050
140050
(14442, 8346)


In [53]:
ingred_counts = zip(cv2.get_feature_names(), np.asarray(recipe_ingredient_matrix2.sum(axis=0)).ravel())

df_ingred_counts = pd.DataFrame(ingred_counts, columns=['ingredient', 'count'])

In [54]:
from IPython.display import display
display(df_ingred_counts.sort(columns='count', ascending=False))

/home/apanza/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,ingredient,count
6392,salt,5688
2930,garlic,4670
1104,butter,3880
8121,white sugar,3640
4936,onion,3511
7954,water,3365
2799,flour,3006
4913,olive oil,2859
718,black pepper,2686
2539,eggs,2124


In [55]:
# https://stackoverflow.com/a/8980156/2491761
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )


In [56]:
save_sparse_csr("ethnic_sparse_recipe_ingredient_matrix", recipe_ingredient_matrix2)

In [57]:
!aws s3 cp ethnic_sparse_recipe_ingredient_matrix.npz s3://RecipeVectors

upload: ./ethnic_sparse_recipe_ingredient_matrix.npz to s3://RecipeVectors/ethnic_sparse_recipe_ingredient_matrix.npz


In [58]:
ingred_cooccurrence_matrix = (recipe_ingredient_matrix2.T * recipe_ingredient_matrix2)

print ingred_cooccurrence_matrix.nnz
print ingred_cooccurrence_matrix.size
print ingred_cooccurrence_matrix.shape

412572
412572
(8346, 8346)


In [60]:
save_sparse_csr('ethnic_ingred_cooccurrence_matrix', ingred_cooccurrence_matrix) #csc matrix also apparently works
!aws s3 cp ethnic_ingred_cooccurrence_matrix.npz s3://RecipeVectors

upload: ./ethnic_ingred_cooccurrence_matrix.npz to s3://RecipeVectors/ethnic_ingred_cooccurrence_matrix.npz


In [61]:
embedding_size = 200
svd2 = TruncatedSVD(n_components=embedding_size)
reduced_cooccurrence_matrix = svd2.fit_transform(ingred_cooccurrence_matrix)

print type(reduced_cooccurrence_matrix)
print reduced_cooccurrence_matrix.size
print reduced_cooccurrence_matrix.shape

<type 'numpy.ndarray'>
1669200
(8346, 200)


In [62]:
with open('ethnic_reduced_cooccurrence_matrix.pkl', 'wb') as f:
    pickle.dump(reduced_cooccurrence_matrix, f)
    
!aws s3 cp ethnic_reduced_cooccurrence_matrix.pkl s3://RecipeVectors

upload: ./ethnic_reduced_cooccurrence_matrix.pkl to s3://RecipeVectors/ethnic_reduced_cooccurrence_matrix.pkl


In [64]:
print 'Unique Ingredients: ',len(unique_vocab)
print 'Embedding matrix shape: ',reduced_cooccurrence_matrix.shape

 Unique Ingredients:  8346
Embedding matrix shape:  (8346, 200)


In [67]:
def createIndexMappingAndWordVec():
    word_to_idx_mapping = {}
    word_vec = {}
    for idx, word in enumerate(unique_vocab):
        word_to_idx_mapping[word] = idx
        word_vec[word] = reduced_cooccurrence_matrix[idx]
    return word_to_idx_mapping, word_vec

word_to_idx_mapping, word_vec = createIndexMappingAndWordVec()

# test the embedding is working:
print 'Index for ingredient(olive oil):',  word_to_idx_mapping['olive oil']
print 'embedding vector for ingredient(olive oil): ', word_vec['olive oil']

Index for ingredient(olive oil): 4913
embedding vector for ingredient(olive oil):  [  3.34271840e+03   2.22598671e+03  -1.13405676e+03  -4.65672759e+02
   1.22262850e+03   7.89527476e+02   1.13468168e+03  -8.09016961e+01
  -4.04061488e+02   3.68470512e+02  -1.66500720e+00   3.54354311e+02
  -3.89987961e+02   9.47744654e+01   7.26336331e+01  -4.02413321e+01
  -6.75791265e+01   1.53491514e+02  -1.10066936e+02  -7.25038080e+01
  -7.72461396e+01  -5.07103168e+01   1.17311259e+01  -2.91898635e+01
   2.15789085e+01   1.01144422e+02   5.86735584e+00   6.27247270e+00
   3.70561142e+01  -2.88767843e+01   2.28968280e+01   3.83165513e+01
   4.84068228e+00  -1.79518064e+01   1.26707851e+01  -1.30573761e+01
  -9.03036712e+00   8.77083773e+00  -3.48782207e+00   6.41965215e+00
  -4.73798866e+00  -1.91367129e+01  -7.82186640e+00   1.07712031e+01
   1.71912812e+00  -4.16118767e+00  -8.18568985e+00  -1.61115375e+00
   1.65786319e+00  -1.57845582e+01   5.50801182e+00  -9.48709681e-01
   1.94421498e+00  -

In [68]:
# for each ingredient in the ingredient list of a recipe, 
# map the ingredient to its corresponding index in the word_to_idx_mapping
def parseDataset(ingredientList):
    ingredientsIndexList = []
    for ingredient in ingredientList:
        try:
            ingredientsIndexList.append(word_to_idx_mapping[ingredient])
        except:
            ingredientsIndexList.append(0)
    return ingredientsIndexList

df['cleanedIngredientsIndexedList'] = df['cleanedIngredients'].apply(parseDataset)

In [70]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['consolidated_category'])
print le.classes_

['african' 'east_asian' 'east_european' 'latin_american' 'middle_eastern'
 'north_american' 'north_european' 'south_asian' 'south_european'
 'southeast_asian']


In [72]:
df['features'] = df['cleanedIngredientsIndexedList']
df['label'] = le.transform(df['consolidated_category'])

df_final = df[['features','label']]

df_final.sample(5)

,features,label
12269,"[1457, 4936, 1815, 7537, 5269, 5104]",4
12038,"[8332, 4463, 7553, 5331, 1005, 3974, 3975]",8
3809,"[1196, 369, 6392, 1104, 8121, 2539, 7773, 4668...",5
14306,"[2799, 6392, 1104, 1994, 7917, 3444, 1995]",2
13907,"[8166, 1465, 7954, 1976, 4936, 8203, 5588, 296...",2


In [84]:
# test the encoding works
le.inverse_transform([0,1,2,3,4,5,6,7,8,9])

array(['african', 'east_asian', 'east_european', 'latin_american',
       'middle_eastern', 'north_american', 'north_european', 'south_asian',
       'south_european', 'southeast_asian'], dtype=object)

In [74]:
print len(word_to_idx_mapping)

8346


In [75]:
# define the variables
VOCAB_SIZE = len(word_to_idx_mapping)
EMBEDDING_SIZE = embedding_size
MAX_LEN = embedding_size

In [76]:
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

features = np.array([np.array(row) for row in df_final['features'].values])
labels = df_final['label'].values

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20)
X_train = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

print 'X_train shape:', X_train.shape
print 'y_train shape:', y_train.shape
print 'X_test shape:', X_test.shape
print 'y_test shape:', y_test.shape

X_train shape: (11553, 200)
y_train shape: (11553,)
X_test shape: (2889, 200)
y_test shape: (2889,)


In [79]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.utils import to_categorical


def trainKerasModel_conv(X,Y):
    print('Shape of data tensor:', X.shape)
    print('Shape of label tensor:', Y.shape)

    print('Training model.')
    # train a 1D convnet with global maxpooling
    sequence_input = Input(shape=(MAX_LEN,), dtype='int32')
#     embedding_layer = Embedding(VOCAB_SIZE,
#                         EMBEDDING_SIZE,
#                         weights=[reduced_cooccurrence_matrix],
#                         input_length=MAX_LEN,
#                         trainable=False)
    embedding_layer = Embedding(VOCAB_SIZE,
                               EMBEDDING_SIZE,
                               input_length=MAX_LEN)
    embedded_sequences = embedding_layer(sequence_input)
    x = Conv1D(128, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Conv1D(128, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    preds = Dense(len(le.classes_), activation='softmax')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
    print model.summary()
    model.fit(X, Y, epochs=10, batch_size=MAX_LEN)
    # Final evaluation of the model
    return model

conv_model = trainKerasModel_conv(X_train, y_train)

('Shape of data tensor:', (11553, 200))
('Shape of label tensor:', (11553,))
Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 200)          1669200   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 196, 128)          128128    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4992)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               639104    
_________________________________________________

In [80]:
def saveKerasModel(modelName, model):
    model_json = model.to_json()
    with open(modelName, "w") as json_file:
        json_file.write(model_json)
def loadKerasModel(modelName):
    json_file = open(modelName, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [81]:
saveKerasModel('cnn_model_ethnic_category.json', conv_model)

In [82]:
y_test_preds = conv_model.predict(X_test)

In [83]:
def print_metrics(true_y, predicted_y, target_names, y_score=None):
    """ Prints classification metrics
    Args:
        true_y: The ground truth target labels
        predicted_y: The predicted labels from the classifier
        y_score: If not None, this is vector of probability scores for positive class (used for roc_curve) (Optional)
    Returns:
        None
    """
    print classification_report(true_y, predicted_y, target_names=target_names)
    cm = confusion_matrix(true_y, predicted_y)
    print "Confusion matrix:"
    print cm
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print
    print "Confusion matrix(normalized):"
    print cm_normalized
    print
    
    print 'Overall accuracy: {}'.format(accuracy_score(true_y, predicted_y))
    print
    if y_score is not None:
        print "Area Under the ROC Curve: {}".format(roc_auc_score(true_y, y_score))
        print

In [94]:
#print_metrics(y_test, y_test_preds, le.classes_)
#print y_test_preds

#print y_test_preds.shape
#print np.argmax(y_test_preds, axis=1).shape
pred_classes = np.argmax(y_test_preds, axis=1)
print_metrics(y_test, pred_classes, le.classes_)

                 precision    recall  f1-score   support

        african       0.28      0.17      0.21        29
     east_asian       0.67      0.67      0.67       144
  east_european       0.20      0.11      0.14        46
 latin_american       0.70      0.68      0.69       505
 middle_eastern       0.46      0.38      0.42       117
 north_american       0.60      0.62      0.61       931
 north_european       0.38      0.43      0.41       354
    south_asian       0.79      0.68      0.73       151
 south_european       0.65      0.66      0.66       516
southeast_asian       0.59      0.55      0.57        96

    avg / total       0.60      0.60      0.60      2889

Confusion matrix:
[[  5   0   0   6   4   7   4   2   0   1]
 [  0  97   0   2   1  24   9   1   1   9]
 [  0   4   5   0   1  20  12   0   4   0]
 [  4   3   1 343   4 106  16   2  23   3]
 [  2   2   0   7  45  13  12   6  29   1]
 [  2  16  10  76  16 581 133   5  85   7]
 [  1   8   8  20   5 117 153   2  34